In [71]:
import os
import random
import math
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from mplfinance.original_flavor import candlestick_ohlc
import matplotlib.dates as mpl_dates

### EMA Bounce Success

In [112]:
#================================================================
# For each timeframe, what is the percent chance that we will see an EMA bounce?

# *** Setup ***
# X previous candles are above/below X ema

# *** Trigger ***
# When EMAs are stacked and low crosses EMA to the downside
# When EMAs are stacked and high crosses EMA to the upside

# *** Continuation ***
# Current+4 high/low is bigger than current close

#================================================================


# Setup - Check for trend with candles below ema
def has_trend(df, current, trend_type, ema, lookback):
    is_trend = False
    
    if trend_type == "up":
        for i in range(current-lookback,current+1):
            if df['close'][i] < df['ema'+ema[0]][i]:
                break
            if i == current and df['close'][current-1] > df['close'][current-lookback]:
                is_trend = True
    if trend_type == "down":
        for i in range(current-lookback,current+1):
            if df['close'][i] > df['ema'+ema[0]][i]:
                break
            if i == current and df['close'][current-1] < df['close'][current-lookback]:
                is_trend = True

    return is_trend


# Trigger - Check for cross of EMA
def has_trigger(df, current, trend_type, ema, lookback):
    is_trigger = False
    
    # When EMAs are stacked and low crosses EMA to the downside
    if trend_type == "up":
        if df['ema'+ema[0]][current] > df['ema'+ema[1]][current] and df['low'][current] < df['ema'+ema[0]][current]:
            is_trigger = True

    # When EMAs are stacked and high crosses EMA to the upside
    if trend_type == "down":
        if df['ema'+ema[0]][current] < df['ema'+ema[1]][current] and df['high'][current] > df['ema'+ema[0]][current]:
            is_trigger = True
    
    return is_trigger
            
    
# Continuation - Check for candles continuing trend. Future Low/High showed a bigger correlation than Close.
def continuation(df, current, trend_type, ema, lookback):    
    is_continuation = False
    
    # Check if future candle high is higher than current close
    if trend_type == "up":
        if df['high'][current+4] > df['close'][current]:
            is_continuation = True
    # Check if future candle low is lower than current close
    if trend_type == "down":
        if df['low'][current+4] < df['close'][current]:
            is_continuation = True
        
    return is_continuation


# Calculate the percent chance of bounce success
def calculate_percent(df):

    lookback = 8
    emas = [["9","21"],["21","200"]]
    for ema in emas:
        bounce_results = {"up_bounce":0,"up_no_bounce":0,"up_bounce_times":[],"down_bounce":0,"down_no_bounce":0,"down_bounce_times":[]}
        for i in range(lookback,df.shape[0]-lookback):

            # Check if setup is correct with up trend and test of lower ema
            if has_trend(df,i,"up",ema,lookback) and has_trigger(df,i,"up",ema,lookback):
                # Check if we see bounce up with future trend continuing
                if continuation(df,i,"up",ema,lookback):    
                    bounce_results["up_bounce"] += 1
                    bounce_results["up_bounce_times"].append(df['time'][i])
                else:
                    bounce_results["up_no_bounce"] += 1 

            # Check if setup is correct with down trend and test of upper ema
            if has_trend(df,i,"down",ema,lookback) and has_trigger(df,i,"down",ema,lookback):
                # Check if we see bounce down with future trend continuing
                if continuation(df,i,"down",ema,lookback):    
                    bounce_results["down_bounce"] += 1
                    bounce_results["down_bounce_times"].append(df['time'][i])
                else:
                    bounce_results["down_no_bounce"] += 1 


        total_up_bounce = bounce_results["up_bounce"] + bounce_results["up_no_bounce"]
        total_down_bounce = bounce_results["down_bounce"] + bounce_results["down_no_bounce"]
            
        up_bounce_percent = round(bounce_results["up_bounce"] / total_up_bounce * 100,2)
        up_no_bounce_percent = round(100 - up_bounce_percent,2)

        down_bounce_percent = round(bounce_results["down_bounce"] / total_down_bounce * 100,2)
        down_no_bounce_percent = round(100 - down_bounce_percent,2)

        print('{}/{} EMA || UT:{} DT:{} || Up_Bounce: {}% Up_No_Bounce: {}% || Down_Bounce: {}% Down_No_Bounce: {}%'.format(ema[0], ema[1], total_up_bounce, total_down_bounce, up_bounce_percent, up_no_bounce_percent, down_bounce_percent, down_no_bounce_percent))
        #print(bounce_results["up_bounce_times"][-3:])
        #print(bounce_results["down_bounce_times"][-3:])



# Data exported from TradingView
stock_list = ["spy-1m.csv","spy-5m.csv","spy-15m.csv","spy-1h.csv","spy-1d.csv"]
#stock_list = ["aapl-1m.csv","aapl-5m.csv","aapl-1h.csv","aapl-1d.csv"]
#stock_list = ["tsla-1m.csv","tsla-5m.csv","tsla-15m.csv","tsla-1h.csv","tsla-1d.csv"]

# Loop through each symbol
for symbol in stock_list:

    print("=======================================")
    df = pd.read_csv(symbol).dropna()
    print("Dataset: " + symbol + " Size: " + str(len(df.index)) + " StartDate: " + df["time"].iloc[0])
    print("")
    df.rename(columns={'EMA': 'ema200', 'EMA.1': 'ema21','EMA.2': 'ema9'}, inplace=True)
    df.drop(columns=['Smoothing Line','Smoothing Line.1','Smoothing Line.2'], inplace=True)
    df = df.reset_index(drop=True)
    calculate_percent(df)




Dataset: spy-1m.csv Size: 10327 StartDate: 2022-12-12T17:53:00Z

9/21 EMA || UT:190 DT:186 || Up_Bounce: 56.32% Up_No_Bounce: 43.68% || Down_Bounce: 57.53% Down_No_Bounce: 42.47%
21/200 EMA || UT:51 DT:50 || Up_Bounce: 54.9% Up_No_Bounce: 45.1% || Down_Bounce: 68.0% Down_No_Bounce: 32.0%
Dataset: spy-5m.csv Size: 9837 StartDate: 2022-07-20T17:25:00Z

9/21 EMA || UT:244 DT:208 || Up_Bounce: 63.11% Up_No_Bounce: 36.89% || Down_Bounce: 57.21% Down_No_Bounce: 42.79%
21/200 EMA || UT:54 DT:54 || Up_Bounce: 68.52% Up_No_Bounce: 31.48% || Down_Bounce: 62.96% Down_No_Bounce: 37.04%
Dataset: spy-15m.csv Size: 10061 StartDate: 2021-08-11T18:45:00Z

9/21 EMA || UT:270 DT:261 || Up_Bounce: 65.93% Up_No_Bounce: 34.07% || Down_Bounce: 65.52% Down_No_Bounce: 34.48%
21/200 EMA || UT:50 DT:54 || Up_Bounce: 66.0% Up_No_Bounce: 34.0% || Down_Bounce: 57.41% Down_No_Bounce: 42.59%
Dataset: spy-1h.csv Size: 10439 StartDate: 2017-02-14T14:30:00Z

9/21 EMA || UT:330 DT:193 || Up_Bounce: 66.97% Up_No_Bounce: 3